In [ ]:
#Modules de modification des données d'entraînement (pour en rajouter artificiellement)
from speechbrain.lobes.augment import TimeDomainSpecAugment
from speechbrain.lobes.augment import EnvCorrupt
#Nous utilisons read_audio qui accepte beaucoup d'extensions pour lire les fichiers audio
from speechbrain.dataio.dataio import read_audio
#Nous utilisons torch afin d'effectuer des opérations sur les fichiers sons (renversement de l'enregistrement, ...)
import torch
import torchaudio
#Modules usuels
import numpy as np
import pandas as pd

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [ ]:
#!pip install speechbrain

# Modification des pistes audio

In [ ]:
##Fonction corrupter : prend le fichier audio "audio_file" et sauvegarde le fichier "déformé" sous le nom save_name
#Nous utilisons deux modules de Speech Augmentation afin de déformer les fichiers audios (cela afin d'ajouter des échantillons artificiels dans l'ensemble de training) :
#  - TimeDomainSpecAugment, qui accélère, ralentit et fait des coupures aléatoires sur un fichier son ;
#  - EnvCorrupt, qui ajoute du bruit de fond aux enregistrement.

def corrupter(audio_file,
              save_name):

    signal = read_audio(audio_file)
    clean = signal.unsqueeze(0)
    # Modifications temporelles aléatoires de l'enregistrement
    do_augment = TimeDomainSpecAugment(speeds=[i for i in range(85, 116)],
                                    perturb_prob=1, 
                                    drop_freq_prob=1, 
                                    drop_chunk_prob=1,
                                    drop_chunk_length_low=1000,
                                    drop_freq_count_low=0,
                                    drop_freq_count_high=np.random.randint(50, size=1)[0],
                                    drop_chunk_count_low=0,
                                    drop_chunk_count_high=np.random.randint(50, size=1)[0],
                                    drop_chunk_length_high=5000)
    # Ajout de bruits de fond
    do_augment_env = EnvCorrupt(reverb_prob=1.0,
                              noise_prob=1.0,
                              noise_num_workers=np.random.randint(100, size=1)[0]
                              )
    
    # Fait passer le fichier audio_file par la pipeline ci-dessus
    augmented_signal = do_augment(clean, torch.ones(1))
    augmented_signal_env = do_augment_env(augmented_signal, torch.ones(1))
    # Sauvegarde du fichier sous le nom "save_name"
    torchaudio.save(save_name, augmented_signal_env, sample_rate=16000)

In [ ]:
# "tab" est le .csv des labels, répartis en un ensemble de Train et un ensemble de Test obtenus par RepeatedKFold
labels=pd.read_csv('SPEAKERS_TYPE.csv',';')
labels.groupby('ORIGIN').count()

C:\Users\Paul\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,ID,Name,SEX
ORIGIN,,,
TEST,50,50,50
TRAIN,201,201,201


In [ ]:
#tableau des labels appartenant à train
train_labels=labels[labels['ORIGIN']=='TRAIN']

In [ ]:
train_labels.sort_values(by='ID')

,ID,Name,ORIGIN,SEX
0,19,LibriSpeech/19-198-0000.flac,TRAIN,F
1,26,LibriSpeech/26-495-0000.flac,TRAIN,M
2,27,LibriSpeech/27-123349-0000.flac,TRAIN,M
3,32,LibriSpeech/32-21625-0000.flac,TRAIN,F
4,40,LibriSpeech/40-121026-0000.flac,TRAIN,F
...,...,...,...,...
196,8747,LibriSpeech/8747-293952-0000.flac,TRAIN,M
197,8770,LibriSpeech/8770-295462-0000.flac,TRAIN,M
198,8797,LibriSpeech/8797-294123-0000.flac,TRAIN,M
199,8838,LibriSpeech/8838-298545-0000.flac,TRAIN,M


In [ ]:
# On utilise la fonction corrupter sur les données de training, pour en augmenter le nombre artificiellement
n0=labels['ID'].max()+1
for i in range(len(train_labels['Name'])):
    save_n='LibriSpeech/'+str(n0+i)+'.flac'
    corrupter(train_labels['Name'].iloc[i],save_n)
    L=pd.DataFrame([[n0+i,save_n,'TRAIN',train_labels['SEX'].iloc[i]]],columns=['ID','Name','ORIGIN','SEX'])
    train_labels=train_labels.append(L,ignore_index=True)

In [ ]:
labels=labels[labels['ORIGIN']=='TEST'].append(train_labels)

In [ ]:
labels.to_csv('SPEAKERS_TYPE_extended.csv',';',index=False) # Nouveau tableau avec les ID des sons déformés ajoutés dans l'ensemble de training

## Inversion des pistes audios pour rajouter des audios

In [ ]:
labels=pd.read_csv('SPEAKERS_TYPE_extended.csv',';')
train_labels=labels[labels['ORIGIN']=='TRAIN']

C:\Users\Paul\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,ID,Name,ORIGIN,SEX
50,19,LibriSpeech/19-198-0000.flac,TRAIN,F
51,26,LibriSpeech/26-495-0000.flac,TRAIN,M
52,27,LibriSpeech/27-123349-0000.flac,TRAIN,M
53,32,LibriSpeech/32-21625-0000.flac,TRAIN,F
54,40,LibriSpeech/40-121026-0000.flac,TRAIN,F
...,...,...,...,...
447,9172,LibriSpeech/9172.flac,TRAIN,M
448,9173,LibriSpeech/9173.flac,TRAIN,M
449,9174,LibriSpeech/9174.flac,TRAIN,M
450,9175,LibriSpeech/9175.flac,TRAIN,M


In [ ]:
##Fonction reverser : prend le fichier audio "audio_file" et sauvegarde le fichier mis à l'envers sous le nom save_name

def reverser(audio_file,
             save_name):
    audio_normal = read_audio(audio_file)
    clean = audio_normal.unsqueeze(0)
    # Prend le tenseur torch correspondant à clean, et le retourne
    rTensor = torch.fliplr(clean)
    # Sauvegarde le fichier audio retourné
    torchaudio.save(save_name, rTensor, sample_rate=16000)

In [ ]:
# Ajout des nouveaux échantillons dans la base de training

n0=labels['ID'].max()+1
for i in range(len(train_labels['Name'])):
    save_n='LibriSpeech/'+str(n0+i)+'.flac'
    reverser(train_labels['Name'].iloc[i],save_n)
    L=pd.DataFrame([[n0+i,save_n,'TRAIN',tab2['SEX'].iloc[i]]],columns=['ID','Name','ORIGIN','SEX'])
    train_labels=train_labels.append(L,ignore_index=True)

In [ ]:
labels=labels[labels['ORIGIN']=='TEST'].append(train_labels)

In [ ]:
labels.to_csv('SPEAKERS_TYPE_extended.csv',';',index=False) # On quadruple ainsi artificiellement l'ensemble de train